In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from def_dict import *
import tensorflow as tf
import pandas as pd

In [2]:
data_path = 'data/celeba_split/'
data_list = os.listdir(data_path)
print(data_list)

['celeba_140000.npy', 'celeba_60000.npy', 'celeba_40000.npy', 'celeba_160000.npy', 'celeba_200000.npy', 'celeba_80000.npy', 'celeba_120000.npy', 'celeba_100000.npy', 'celeba_180000.npy', 'celeba_20000.npy']


In [3]:
dataset = [np.load(data_path+data) for data in data_list[:4]]
x_data = np.concatenate(dataset)
x_data.shape

(80000, 64, 64, 3)

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_valid = train_test_split(x_data, train_size = 0.9)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, x_train)).shuffle(10000).batch(256)
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, x_valid)).shuffle(10000).batch(256)

In [10]:
from build_models import build_vae

encoder, decoder, vae = build_vae(x_data, 100)
#encoder.summary()
#decoder.summary()
vae.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
encoder (Model)              [(None, 100), (None, 100) 277160    
_________________________________________________________________
decoder (Model)              (None, 64, 64, 3)         251843    
Total params: 529,003
Trainable params: 529,003
Non-trainable params: 0
_________________________________________________________________


In [16]:
optimizer = tf.keras.optimizers.Adam(0.001)
train_loss = tf.keras.metrics.Mean(name='train_loss')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')

def get_rec_loss(inputs, predictions):
    rec_loss = tf.keras.losses.binary_crossentropy(inputs, predictions)
    rec_loss = tf.reduce_mean(rec_loss)
    rec_loss *= 64*64
    return rec_loss

def get_kl_loss(z_log_var, z_mean):
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = tf.reduce_mean(kl_loss)
    kl_loss *= -0.5
    
    return kl_loss

In [17]:
from tensorflow.keras import backend as K

@tf.function
def train_step(inputs):
    with tf.GradientTape() as tape:
        
        # Get model ouputs
        z_log_var, z_mean, z = encoder(inputs)
        predictions = decoder(z)
        
        # Compute losses
        rec_loss = get_rec_loss(inputs, predictions)
        kl_loss = get_kl_loss(z_log_var, z_mean)
        loss = rec_loss + kl_loss
    
    # Compute gradients
    varialbes = vae.trainable_variables
    gradients = tape.gradient(loss, varialbes)
    # Update weights
    optimizer.apply_gradients(zip(gradients, varialbes))
    
    # Update train loss
    train_loss(loss)

@tf.function
def valid_step(inputs):
    with tf.GradientTape() as tape:
        
        # Get model ouputs without training
        z_log_var, z_mean, z = encoder(inputs, training=False)
        predictions = decoder(z, training=False)
        
        # Compute losses
        rec_loss = get_rec_loss(inputs, predictions)
        kl_loss = get_kl_loss(z_log_var, z_mean)
        loss = K.mean(rec_loss + kl_loss)
    
    # Update valid loss 
    valid_loss(loss)

In [19]:
save_dir = 'results_VAE/train_10'
if_not_make(save_dir)

def check_recimg(save_dir, epoch):
    org_img = x_train[:100]
    rec_img = vae(x_train[:100], training=False)
    fig, ax = plt.subplots(6, 10, figsize=(20, 10))
    for i in range(10):
        for j in range(6):
            if j%2 ==0: img=org_img 
            else: img=rec_img
            ax[j][i].set_axis_off()
            ax[j][i].imshow(img[10*(j//2)+i])
    
    plt.savefig('%s/recimg_%i.png'%(save_dir,epoch))
    plt.close('all')
    
def reduce_lr(pre_v_loss, v_loss, count, lr, patience, factor, min_lr):
    if v_loss < pre_v_loss:
        count = 0
    else:
        count += 1
        if count >= patience: 
            lr = lr*factor
            if lr < min_lr: 
                lr = min_lr
            count = 0
            print('reduce learning rate..', lr)    
    return count, lr

checkpoint = tf.train.Checkpoint(step=tf.Variable(1), encoder=encoder, decoder=decoder, vae=vae)
csv_logger = tf.keras.callbacks.CSVLogger(save_dir+'training.log')

In [20]:
epochs = 5000

# Initialize values
best_v_loss, count = float('inf'), 0

# Start epoch loop
for epoch in range(epochs):
    for inputs, outputs in train_ds:
        train_step(inputs)
    for inputs, outputs in valid_ds:
        valid_step(inputs)
    
    # Get loss and leraning rate at this epoch
    t_loss = train_loss.result().numpy() 
    v_loss = valid_loss.result().numpy()
    l_rate = optimizer.learning_rate.numpy()

    # Plot reconstruct image per 10 epochs
    if epoch%10 ==0:
        check_recimg(save_dir, epoch)
    
    # Control learning rate
    count, lr  = reduce_lr(best_v_loss, v_loss, count, l_rate, 5, 0.2, 0.00001)
    optimizer.learning_rate = lr
    
    # Save checkpoint if best v_loss 
    if v_loss < best_v_loss:
        best_v_loss = v_loss
        checkpoint.save(file_prefix=os.path.join(save_dir, 'ckp'))
    
    # Save loss, lerning rate
    print("* %i * loss: %f, v_loss: %f, best_v_loss: %f, l_rate: %f, lr_count: %i"%(epoch, t_loss, v_loss, best_v_loss, l_rate, count ))
    df = pd.DataFrame({'epoch':[epoch], 'loss':[t_loss], 'v_loss':[v_loss],'best_v_loss':[best_v_loss], 'l_rate':[l_rate]  } )
    df.to_csv(save_dir+'/process.csv', mode='a', header=False)
    
    # Reset loss
    train_loss.reset_states()
    valid_loss.reset_states()

* 0 * loss: 2190.361816, v_loss: 2193.491943, best_v_loss: 2193.491943, l_rate: 0.000010, lr_count: 0
* 1 * loss: 2190.460693, v_loss: 2190.596924, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 0
* 2 * loss: 2190.405029, v_loss: 2191.945312, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 1
* 3 * loss: 2190.347168, v_loss: 2191.830322, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 2
* 4 * loss: 2190.188477, v_loss: 2191.884521, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 3
* 5 * loss: 2190.252930, v_loss: 2191.272705, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 6 * loss: 2190.318115, v_loss: 2191.463135, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 0
* 7 * loss: 2190.411621, v_loss: 2192.399658, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 1
* 8 * loss: 2190.450928, v_loss: 2191.979980, best_v_loss: 2190.596924, l_rate: 0.000010, lr_count: 2
* 9 * loss: 2190.332031, v_loss: 2192.627930, best_v_

* 76 * loss: 2190.046631, v_loss: 2191.819092, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 3
* 77 * loss: 2190.187744, v_loss: 2191.550293, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 78 * loss: 2190.181641, v_loss: 2192.433838, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 0
* 79 * loss: 2190.176758, v_loss: 2192.123779, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 1
* 80 * loss: 2190.198242, v_loss: 2192.180664, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 2
* 81 * loss: 2190.053955, v_loss: 2190.350098, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 3
* 82 * loss: 2190.052979, v_loss: 2193.152100, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 83 * loss: 2190.285156, v_loss: 2190.833740, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 0
* 84 * loss: 2190.065430, v_loss: 2191.512451, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 1
* 85 * loss: 21

* 151 * loss: 2189.906738, v_loss: 2191.632812, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 3
* 152 * loss: 2189.998779, v_loss: 2191.432129, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 153 * loss: 2189.842773, v_loss: 2191.096680, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 0
* 154 * loss: 2189.829346, v_loss: 2190.799316, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 1
* 155 * loss: 2189.929199, v_loss: 2190.997314, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 2
* 156 * loss: 2190.064941, v_loss: 2192.169434, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 3
* 157 * loss: 2190.031006, v_loss: 2190.198975, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 158 * loss: 2189.866699, v_loss: 2191.514160, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 0
* 159 * loss: 2189.813721, v_loss: 2190.611084, best_v_loss: 2190.022461, l_rate: 0.000010, lr_count: 1
* 160 

* 226 * loss: 2189.710693, v_loss: 2191.052002, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 2
* 227 * loss: 2189.685303, v_loss: 2191.433838, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 3
* 228 * loss: 2189.682617, v_loss: 2191.911377, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 229 * loss: 2189.695801, v_loss: 2191.555176, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 0
* 230 * loss: 2189.578125, v_loss: 2191.122559, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 1
* 231 * loss: 2189.684570, v_loss: 2191.150146, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 2
* 232 * loss: 2189.390869, v_loss: 2191.945557, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 3
* 233 * loss: 2189.554443, v_loss: 2190.300293, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 234 * loss: 2189.628906, v_loss: 2191.223389, best_v_loss: 2189.657959, l_rate: 0.000010, lr_count: 0
* 235 

reduce learning rate.. 1e-05
* 301 * loss: 2189.527588, v_loss: 2190.647949, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 0
* 302 * loss: 2189.464355, v_loss: 2191.413574, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 1
* 303 * loss: 2189.495850, v_loss: 2191.701660, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 2
* 304 * loss: 2189.429443, v_loss: 2189.777344, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 3
* 305 * loss: 2189.466309, v_loss: 2190.704590, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 306 * loss: 2189.264404, v_loss: 2191.489746, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 0
* 307 * loss: 2189.413574, v_loss: 2191.435303, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 1
* 308 * loss: 2189.500488, v_loss: 2191.312012, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 2
* 309 * loss: 2189.419922, v_loss: 2191.615723, best_v_loss: 2189.341064, l_rate: 0.000010, lr_count: 3
* 310 

...

* 4634 * loss: 2184.828857, v_loss: 2185.365723, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 1
* 4635 * loss: 2184.704346, v_loss: 2186.290771, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 2
* 4636 * loss: 2184.773926, v_loss: 2187.308350, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 3
* 4637 * loss: 2184.725830, v_loss: 2185.555176, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 4638 * loss: 2184.657471, v_loss: 2187.839355, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 0
* 4639 * loss: 2184.677734, v_loss: 2187.127197, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 1
* 4640 * loss: 2184.704346, v_loss: 2187.124023, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 2
* 4641 * loss: 2184.712646, v_loss: 2186.858887, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 3
* 4642 * loss: 2184.741943, v_loss: 2187.361328, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-

* 4709 * loss: 2184.642822, v_loss: 2187.248779, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 1
* 4710 * loss: 2184.637207, v_loss: 2186.151123, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 2
* 4711 * loss: 2184.624512, v_loss: 2187.339355, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 3
* 4712 * loss: 2184.887207, v_loss: 2187.328857, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 4713 * loss: 2184.732910, v_loss: 2187.373779, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 0
* 4714 * loss: 2184.575439, v_loss: 2187.469727, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 1
* 4715 * loss: 2184.647705, v_loss: 2186.869873, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 2
* 4716 * loss: 2184.772705, v_loss: 2186.295410, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 3
* 4717 * loss: 2184.624756, v_loss: 2186.665771, best_v_loss: 2184.489258, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-

* 4784 * loss: 2184.771484, v_loss: 2186.020752, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 3
* 4785 * loss: 2184.597656, v_loss: 2186.421387, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 4786 * loss: 2184.724121, v_loss: 2185.062744, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 0
* 4787 * loss: 2184.638184, v_loss: 2186.808838, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 1
* 4788 * loss: 2184.756104, v_loss: 2186.961182, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 2
* 4789 * loss: 2184.671387, v_loss: 2186.950195, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 3
* 4790 * loss: 2184.670898, v_loss: 2187.427490, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 4791 * loss: 2184.692139, v_loss: 2187.311768, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 0
* 4792 * loss: 2184.610352, v_loss: 2187.944092, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count:

* 4858 * loss: 2184.682129, v_loss: 2186.721191, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 2
* 4859 * loss: 2184.716797, v_loss: 2185.909668, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 3
* 4860 * loss: 2184.697266, v_loss: 2187.660645, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 4861 * loss: 2184.817383, v_loss: 2187.300781, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 0
* 4862 * loss: 2184.769775, v_loss: 2186.532715, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 1
* 4863 * loss: 2184.648682, v_loss: 2185.927979, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 2
* 4864 * loss: 2184.805176, v_loss: 2185.682129, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 3
* 4865 * loss: 2184.657715, v_loss: 2186.215088, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 4866 * loss: 2184.664307, v_loss: 2186.520508, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count:

* 4932 * loss: 2184.772949, v_loss: 2186.343262, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 1
* 4933 * loss: 2184.771240, v_loss: 2185.659912, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 2
* 4934 * loss: 2184.604980, v_loss: 2187.802246, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 3
* 4935 * loss: 2184.764404, v_loss: 2186.631836, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-05
* 4936 * loss: 2184.899658, v_loss: 2187.058838, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 0
* 4937 * loss: 2184.760742, v_loss: 2186.797119, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 1
* 4938 * loss: 2184.733154, v_loss: 2187.355957, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 2
* 4939 * loss: 2184.532715, v_loss: 2186.546875, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 3
* 4940 * loss: 2184.708008, v_loss: 2186.685791, best_v_loss: 2184.468506, l_rate: 0.000010, lr_count: 4
reduce learning rate.. 1e-